# 🎙️ Speech Emotion Recognition - Multi-Dataset Training

This notebook trains emotion recognition models on **multiple datasets** for improved accuracy:
- **RAVDESS** - Ryerson Audio-Visual Database (1,440 files)
- **CREMA-D** - Crowd-sourced Emotional Multimodal Actors Dataset (7,442 files)
- **TESS** - Toronto Emotional Speech Set (2,800 files)
- **SAVEE** - Surrey Audio-Visual Expressed Emotion (480 files)

**Total: ~12,000+ audio samples**

## 1. Setup & Install Dependencies

In [ ]:
# Install required packages
!pip install -q librosa soundfile kaggle tensorflow scikit-learn matplotlib seaborn

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## 2. Download Datasets from Kaggle

Enter your Kaggle credentials below:

In [ ]:
# Enter your Kaggle credentials
KAGGLE_USERNAME = "your_username"  # @param {type:"string"}
KAGGLE_KEY = "your_api_key"  # @param {type:"string"}

# Setup Kaggle
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write(f'{{"username":"{KAGGLE_USERNAME}","key":"{KAGGLE_KEY}"}}')
os.chmod('/root/.kaggle/kaggle.json', 0o600)
print("✓ Kaggle configured")

In [ ]:
# Download datasets
!mkdir -p datasets

# RAVDESS
print("Downloading RAVDESS...")
!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio -p datasets/ravdess --unzip -q

# CREMA-D
print("Downloading CREMA-D...")
!kaggle datasets download -d ejlok1/cremad -p datasets/cremad --unzip -q

# TESS
print("Downloading TESS...")
!kaggle datasets download -d ejlok1/toronto-emotional-speech-set-tess -p datasets/tess --unzip -q

# SAVEE
print("Downloading SAVEE...")
!kaggle datasets download -d ejlok1/surrey-audiovisual-expressed-emotion-savee -p datasets/savee --unzip -q

print("\n✓ All datasets downloaded!")

## 3. Feature Extraction Functions

In [ ]:
# Audio configuration
SAMPLE_RATE = 22050
DURATION = 3  # seconds
N_MELS = 128
N_MFCC = 40
HOP_LENGTH = 512

# Unified emotion labels
EMOTIONS = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']

def extract_mel_spectrogram(file_path, sr=SAMPLE_RATE, duration=DURATION):
    """Extract mel spectrogram from audio file."""
    try:
        y, _ = librosa.load(file_path, sr=sr, duration=duration)
        
        # Pad or trim to fixed length
        target_length = sr * duration
        if len(y) < target_length:
            y = np.pad(y, (0, target_length - len(y)))
        else:
            y = y[:target_length]
        
        # Extract mel spectrogram
        mel_spec = librosa.feature.melspectrogram(
            y=y, sr=sr, n_mels=N_MELS, hop_length=HOP_LENGTH
        )
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        
        return mel_spec_db
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def extract_features(file_path, sr=SAMPLE_RATE, duration=DURATION):
    """Extract combined features (MFCC, Chroma, Mel, Contrast, Tonnetz)."""
    try:
        y, _ = librosa.load(file_path, sr=sr, duration=duration)
        
        # Pad or trim
        target_length = sr * duration
        if len(y) < target_length:
            y = np.pad(y, (0, target_length - len(y)))
        else:
            y = y[:target_length]
        
        # Extract features
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        mel = librosa.feature.melspectrogram(y=y, sr=sr)
        contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
        
        # Aggregate features
        features = np.hstack([
            np.mean(mfcc, axis=1), np.std(mfcc, axis=1),
            np.mean(chroma, axis=1), np.std(chroma, axis=1),
            np.mean(mel, axis=1), np.std(mel, axis=1),
            np.mean(contrast, axis=1), np.std(contrast, axis=1),
            np.mean(tonnetz, axis=1), np.std(tonnetz, axis=1)
        ])
        
        return features
    except Exception as e:
        print(f"Error extracting features from {file_path}: {e}")
        return None

## 4. Dataset Parsers

In [ ]:
def parse_ravdess(base_path):
    """Parse RAVDESS dataset."""
    # RAVDESS emotion mapping: 01=neutral, 02=calm, 03=happy, 04=sad, 05=angry, 06=fearful, 07=disgust, 08=surprised
    emotion_map = {
        '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
        '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
    }
    
    files = []
    for root, dirs, filenames in os.walk(base_path):
        for f in filenames:
            if f.endswith('.wav'):
                parts = f.split('-')
                if len(parts) >= 3:
                    emotion_code = parts[2]
                    if emotion_code in emotion_map:
                        files.append({
                            'path': os.path.join(root, f),
                            'emotion': emotion_map[emotion_code],
                            'dataset': 'RAVDESS'
                        })
    return files

def parse_cremad(base_path):
    """Parse CREMA-D dataset."""
    # CREMA-D emotion mapping
    emotion_map = {
        'ANG': 'angry', 'DIS': 'disgust', 'FEA': 'fearful',
        'HAP': 'happy', 'NEU': 'neutral', 'SAD': 'sad'
    }
    
    files = []
    for root, dirs, filenames in os.walk(base_path):
        for f in filenames:
            if f.endswith('.wav'):
                parts = f.split('_')
                if len(parts) >= 3:
                    emotion_code = parts[2]
                    if emotion_code in emotion_map:
                        files.append({
                            'path': os.path.join(root, f),
                            'emotion': emotion_map[emotion_code],
                            'dataset': 'CREMA-D'
                        })
    return files

def parse_tess(base_path):
    """Parse TESS dataset."""
    emotion_map = {
        'angry': 'angry', 'disgust': 'disgust', 'fear': 'fearful',
        'happy': 'happy', 'neutral': 'neutral', 'ps': 'surprised', 'sad': 'sad'
    }
    
    files = []
    for root, dirs, filenames in os.walk(base_path):
        for f in filenames:
            if f.endswith('.wav'):
                # TESS format: OAF_word_emotion.wav or YAF_word_emotion.wav
                parts = f.replace('.wav', '').split('_')
                if len(parts) >= 2:
                    emotion = parts[-1].lower()
                    if emotion in emotion_map:
                        files.append({
                            'path': os.path.join(root, f),
                            'emotion': emotion_map[emotion],
                            'dataset': 'TESS'
                        })
    return files

def parse_savee(base_path):
    """Parse SAVEE dataset."""
    emotion_map = {
        'a': 'angry', 'd': 'disgust', 'f': 'fearful',
        'h': 'happy', 'n': 'neutral', 'sa': 'sad', 'su': 'surprised'
    }
    
    files = []
    for root, dirs, filenames in os.walk(base_path):
        for f in filenames:
            if f.endswith('.wav'):
                # SAVEE format: DC_a01.wav (emotion code at start after speaker)
                name = f.replace('.wav', '')
                for code, emotion in emotion_map.items():
                    if f'_{code}' in name.lower() or name.lower().startswith(code):
                        files.append({
                            'path': os.path.join(root, f),
                            'emotion': emotion,
                            'dataset': 'SAVEE'
                        })
                        break
    return files

print("✓ Dataset parsers defined")

## 5. Load and Combine All Datasets

In [ ]:
# Parse all datasets
all_files = []

# RAVDESS
ravdess_path = 'datasets/ravdess'
if os.path.exists(ravdess_path):
    ravdess_files = parse_ravdess(ravdess_path)
    all_files.extend(ravdess_files)
    print(f"RAVDESS: {len(ravdess_files)} files")

# CREMA-D
cremad_path = 'datasets/cremad'
if os.path.exists(cremad_path):
    cremad_files = parse_cremad(cremad_path)
    all_files.extend(cremad_files)
    print(f"CREMA-D: {len(cremad_files)} files")

# TESS
tess_path = 'datasets/tess'
if os.path.exists(tess_path):
    tess_files = parse_tess(tess_path)
    all_files.extend(tess_files)
    print(f"TESS: {len(tess_files)} files")

# SAVEE
savee_path = 'datasets/savee'
if os.path.exists(savee_path):
    savee_files = parse_savee(savee_path)
    all_files.extend(savee_files)
    print(f"SAVEE: {len(savee_files)} files")

print(f"\n✓ Total files: {len(all_files)}")

# Create DataFrame
df = pd.DataFrame(all_files)
print(f"\nEmotion distribution:")
print(df['emotion'].value_counts())

In [ ]:
# Visualize dataset distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# By emotion
emotion_counts = df['emotion'].value_counts()
colors = ['#8e8e93', '#5ac8fa', '#ffcc00', '#5856d6', '#ff3b30', '#af52de', '#34c759', '#ff9500']
axes[0].bar(emotion_counts.index, emotion_counts.values, color=colors[:len(emotion_counts)])
axes[0].set_title('Samples by Emotion', fontsize=14)
axes[0].set_xlabel('Emotion')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# By dataset
dataset_counts = df['dataset'].value_counts()
axes[1].pie(dataset_counts.values, labels=dataset_counts.index, autopct='%1.1f%%', 
            colors=['#0a84ff', '#30d158', '#ff9f0a', '#ff3b30'])
axes[1].set_title('Samples by Dataset', fontsize=14)

plt.tight_layout()
plt.savefig('dataset_distribution.png', dpi=150)
plt.show()

## 6. Extract Features from All Files

In [ ]:
from tqdm import tqdm

# Extract mel spectrograms
print("Extracting mel spectrograms...")
mel_features = []
mel_labels = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    mel = extract_mel_spectrogram(row['path'])
    if mel is not None:
        mel_features.append(mel)
        mel_labels.append(row['emotion'])

X_mel = np.array(mel_features)
y_mel = np.array(mel_labels)

print(f"\n✓ Mel spectrograms: {X_mel.shape}")
print(f"✓ Labels: {y_mel.shape}")

In [ ]:
# Extract combined features for LSTM
print("Extracting combined features...")
combined_features = []
combined_labels = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    feat = extract_features(row['path'])
    if feat is not None:
        combined_features.append(feat)
        combined_labels.append(row['emotion'])

X_combined = np.array(combined_features)
y_combined = np.array(combined_labels)

print(f"\n✓ Combined features: {X_combined.shape}")

## 7. Prepare Data for Training

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
label_encoder.fit(EMOTIONS)

y_mel_encoded = label_encoder.transform(y_mel)
y_combined_encoded = label_encoder.transform(y_combined)

print(f"Classes: {label_encoder.classes_}")

# Split data
X_mel_train, X_mel_test, y_mel_train, y_mel_test = train_test_split(
    X_mel, y_mel_encoded, test_size=0.2, random_state=42, stratify=y_mel_encoded
)

X_comb_train, X_comb_test, y_comb_train, y_comb_test = train_test_split(
    X_combined, y_combined_encoded, test_size=0.2, random_state=42, stratify=y_combined_encoded
)

# Normalize combined features
scaler = StandardScaler()
X_comb_train_scaled = scaler.fit_transform(X_comb_train)
X_comb_test_scaled = scaler.transform(X_comb_test)

# Reshape for CNN (add channel dimension)
X_mel_train = X_mel_train[..., np.newaxis]
X_mel_test = X_mel_test[..., np.newaxis]

print(f"\nTraining set: {X_mel_train.shape[0]} samples")
print(f"Test set: {X_mel_test.shape[0]} samples")

## 8. Define Enhanced Model Architectures

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization,
    LSTM, Bidirectional, Input, Reshape, GlobalAveragePooling2D,
    TimeDistributed, Attention, MultiHeadAttention, LayerNormalization
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

num_classes = len(EMOTIONS)

def build_cnn_model(input_shape):
    """Enhanced CNN model with residual connections."""
    model = Sequential([
        # Block 1
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Block 2
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Block 3
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Block 4
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        GlobalAveragePooling2D(),
        
        # Dense layers
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def build_cnn_lstm_model(input_shape):
    """Enhanced CNN-LSTM hybrid model."""
    inputs = Input(shape=input_shape)
    
    # CNN feature extraction
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.25)(x)
    
    # Reshape for LSTM
    shape = x.shape
    x = Reshape((shape[1], shape[2] * shape[3]))(x)
    
    # Bidirectional LSTM
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = Bidirectional(LSTM(64))(x)
    x = Dropout(0.3)(x)
    
    # Dense layers
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def build_lstm_model(input_shape):
    """Enhanced LSTM model for combined features."""
    model = Sequential([
        Reshape((1, input_shape[0]), input_shape=input_shape),
        
        Bidirectional(LSTM(256, return_sequences=True)),
        Dropout(0.3),
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.3),
        Bidirectional(LSTM(64)),
        Dropout(0.3),
        
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

print("✓ Model architectures defined")

## 9. Train Models

In [ ]:
# Training callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
]

EPOCHS = 100
BATCH_SIZE = 32

In [ ]:
# Train CNN Model
print("="*60)
print("Training CNN Model")
print("="*60)

cnn_model = build_cnn_model(X_mel_train.shape[1:])
cnn_model.summary()

cnn_history = cnn_model.fit(
    X_mel_train, y_mel_train,
    validation_split=0.2,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

cnn_score = cnn_model.evaluate(X_mel_test, y_mel_test)
print(f"\nCNN Test Accuracy: {cnn_score[1]*100:.2f}%")

In [ ]:
# Train CNN-LSTM Model
print("="*60)
print("Training CNN-LSTM Model")
print("="*60)

cnn_lstm_model = build_cnn_lstm_model(X_mel_train.shape[1:])
cnn_lstm_model.summary()

cnn_lstm_history = cnn_lstm_model.fit(
    X_mel_train, y_mel_train,
    validation_split=0.2,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

cnn_lstm_score = cnn_lstm_model.evaluate(X_mel_test, y_mel_test)
print(f"\nCNN-LSTM Test Accuracy: {cnn_lstm_score[1]*100:.2f}%")

In [ ]:
# Train LSTM Model
print("="*60)
print("Training LSTM Model")
print("="*60)

lstm_model = build_lstm_model((X_comb_train_scaled.shape[1],))
lstm_model.summary()

lstm_history = lstm_model.fit(
    X_comb_train_scaled, y_comb_train,
    validation_split=0.2,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

lstm_score = lstm_model.evaluate(X_comb_test_scaled, y_comb_test)
print(f"\nLSTM Test Accuracy: {lstm_score[1]*100:.2f}%")

## 10. Evaluate and Compare Models

In [ ]:
# Plot training history (all models)
import os
os.makedirs('plots', exist_ok=True)

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

models_data = [
    ('CNN', cnn_history),
    ('CNN-LSTM', cnn_lstm_history),
    ('LSTM', lstm_history)
]

for i, (name, history) in enumerate(models_data):
    # Accuracy
    axes[0, i].plot(history.history['accuracy'], label='Train')
    axes[0, i].plot(history.history['val_accuracy'], label='Val')
    axes[0, i].set_title(f'{name} Accuracy')
    axes[0, i].legend()
    axes[0, i].set_xlabel('Epoch')

    # Loss
    axes[1, i].plot(history.history['loss'], label='Train')
    axes[1, i].plot(history.history['val_loss'], label='Val')
    axes[1, i].set_title(f'{name} Loss')
    axes[1, i].legend()
    axes[1, i].set_xlabel('Epoch')

plt.tight_layout()

# Save both names used in README (aliasing the same figure)
plt.savefig('plots/all_models_training_history.png', dpi=200)
plt.savefig('plots/training_history.png', dpi=200)
plt.show()

print('✓ Saved:', 'plots/all_models_training_history.png', 'plots/training_history.png')

In [ ]:
# Confusion matrices (all models)
import os
os.makedirs('plots', exist_ok=True)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# CNN
y_pred_cnn = np.argmax(cnn_model.predict(X_mel_test), axis=1)
cm_cnn = confusion_matrix(y_mel_test, y_pred_cnn)
sns.heatmap(cm_cnn, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
axes[0].set_title(f'CNN (Acc: {cnn_score[1]*100:.1f}%)')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# CNN-LSTM
y_pred_cnn_lstm = np.argmax(cnn_lstm_model.predict(X_mel_test), axis=1)
cm_cnn_lstm = confusion_matrix(y_mel_test, y_pred_cnn_lstm)
sns.heatmap(cm_cnn_lstm, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
axes[1].set_title(f'CNN-LSTM (Acc: {cnn_lstm_score[1]*100:.1f}%)')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

# LSTM
y_pred_lstm = np.argmax(lstm_model.predict(X_comb_test_scaled), axis=1)
cm_lstm = confusion_matrix(y_comb_test, y_pred_lstm)
sns.heatmap(cm_lstm, annot=True, fmt='d', cmap='Oranges', ax=axes[2],
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
axes[2].set_title(f'LSTM (Acc: {lstm_score[1]*100:.1f}%)')
axes[2].set_xlabel('Predicted')
axes[2].set_ylabel('Actual')

plt.tight_layout()

# Save both names used in README (aliasing the same figure)
plt.savefig('plots/all_models_confusion_matrices.png', dpi=200)
plt.savefig('plots/confusion_matrix.png', dpi=200)
plt.show()

print('✓ Saved:', 'plots/all_models_confusion_matrices.png', 'plots/confusion_matrix.png')

In [ ]:
# Download plots (Colab)
# This will zip the `plots/` folder and download it.
import os

if not os.path.exists('plots'):
    raise FileNotFoundError("plots/ folder not found. Run the plot-generation cells first.")

!zip -r plots_multi_dataset.zip plots/ -q
print('✓ Created plots_multi_dataset.zip')

try:
    from google.colab import files
    files.download('plots_multi_dataset.zip')
    print('✓ Download started')
except Exception as e:
    print('Not running in Colab (or download not available).')
    print('Zip file saved as: plots_multi_dataset.zip')
    print('Error:', e)

In [ ]:
# Model comparison
print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)
print(f"CNN:      {cnn_score[1]*100:.2f}% accuracy")
print(f"CNN-LSTM: {cnn_lstm_score[1]*100:.2f}% accuracy")
print(f"LSTM:     {lstm_score[1]*100:.2f}% accuracy")
print("="*60)

# Classification reports
print("\nCNN Classification Report:")
print(classification_report(y_mel_test, y_pred_cnn, target_names=label_encoder.classes_))

print("\nCNN-LSTM Classification Report:")
print(classification_report(y_mel_test, y_pred_cnn_lstm, target_names=label_encoder.classes_))

print("\nLSTM Classification Report:")
print(classification_report(y_comb_test, y_pred_lstm, target_names=label_encoder.classes_))

## 11. Save Models

In [ ]:
# Create models directory
!mkdir -p models

# Save models
cnn_model.save('models/emotion_model_cnn.keras')
cnn_lstm_model.save('models/emotion_model_cnn_lstm.keras')
lstm_model.save('models/emotion_model_lstm.keras')

# Save label encoder and scaler
with open('models/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

with open('models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("✓ All models saved!")
print("\nFiles:")
!ls -la models/

## 12. Download Models

In [ ]:
# Zip models for download
!zip -r trained_models_multi_dataset.zip models/

# Download
from google.colab import files
files.download('trained_models_multi_dataset.zip')

print("\n✓ Download complete!")
print("\nTo use these models:")
print("1. Extract the zip file")
print("2. Copy the 'models' folder to your SpeechEmotionRecognition project")
print("3. Run: python3 app.py")